In [4]:
import mysql.connector
import pandas as pd
from datetime import date, datetime
import pprint
import math
import csv

In [5]:
#colocar aqui os dados necessarios para o filtro
ano = "(2022)"
mes = "(2)"
cliente = "(847997, 847995, 847245, 847994, 847996, 847993, 847992)"
#VERIFICAR OS FERIADOS E OS DIAS QUE DEVEM SER ADICIONADOS LA EMBAIXO NO CÓDIGO. 
#PARA MAIS DE UM FERIADO, ADD IFS UM ABAIXO DO OUTRO
#(o comentario na parte do codigo esta igual ao acima)

In [ ]:
#conexão com o banco de dados para capturar as solicitações desejadas
con = mysql.connector.connect(host="#######", 
                              user="#######", passwd="###############", db="#######")
cursor = con.cursor(buffered=True)
query = "SELECT data_solicitacao, data_conclusao, LEFT(f.obs_geral,LOCATE(':',obs_geral) - 1) as tipo_renda FROM fc_ficha as f JOIN fc_entidade as e ON f.eid_cliente = e.eid WHERE f.eid_cliente in "+cliente+" AND MONTH(f.data_solicitacao) in "+mes+" AND YEAR(f.data_solicitacao) IN "+ano+" GROUP BY f.fid ORDER BY data_solicitacao ASC"
cursor.execute(query)

In [ ]:
#criando um dicionário para montagem dos dados
relatorio = dict()

#criando um dicionário para pegar os tempos de resolução da análise
duracoes = {}
duracoes['TOTAL'] = []

In [ ]:
for resultado in cursor:
    
    data_solicitacao = resultado[0]
    data_conclusao = resultado[1]
    tipo_renda = resultado[2]
    
    #existem datas de conclusão nulas. só tem como realizar o cálculo se não for nula  
    if pd.notnull(data_conclusao) or pd.notnull(data_solicitacao): 
            
        if tipo_renda in ['LIMITE DE CARTÃO DE CRÉDITO', 'LIMITE DO CARTÃO DE CRÉDITO', 'RENDA PESSOAL', 'RENDA PRESUMIDA']:
            
            #existem duas formas de escrita para o do limite: 
            #limite DE cartão e limite DO cartão. padronizando para LIMITE CARTÃO DE CRÉDITO
            tipo_renda = 'LIMITE CARTÃO DE CRÉDITO' if (tipo_renda == 'LIMITE DE CARTÃO DE CRÉDITO' 
                                                  or tipo_renda == 'LIMITE DO CARTÃO DE CRÉDITO') else tipo_renda

            #caso não exista a chave com o nome do tipo de renda, criar todos os dados a serem registrados
            if tipo_renda not in relatorio:
                relatorio[tipo_renda] = {}
                relatorio[tipo_renda]['TOTAL'] = 0 
                relatorio[tipo_renda]['TEMPO ATENDIMENTO DIAS'] = {}
                relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS'] = {}
                relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 15'] = 0
                relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 30'] = 0
                relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 40'] = 0
                relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 50'] = 0
                relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 60'] = 0
                relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS']['MAIS DE 60'] = 0
                        
            #pegando a data de inicio da analise e estabelecendo as horas de expediente
            inicio_expediente = data_solicitacao
            inicio_expediente = inicio_expediente.replace(hour = 9)
            inicio_expediente = inicio_expediente.replace(minute = 0)
            inicio_expediente = inicio_expediente.replace(second = 0)

            fim_expediente = data_solicitacao
            fim_expediente = fim_expediente.replace(hour = 17)
            fim_expediente = fim_expediente.replace(minute = 30)
            fim_expediente = fim_expediente.replace(second = 0)

            #pegando a duração de dias e de horas das datas de fim e início da análise
            duracao = data_conclusao - data_solicitacao

            #imprimindo nome e datas para verificação e testes
            print(tipo_renda)
            print("Data e hora de início: ", data_solicitacao)
            print("Data e hora de fim: ", data_conclusao)
            print("Duração: " + str(duracao.days) + " dias / " + str(round(duracao.seconds/60)) + " minutos")        

            # se a data de fim for maior que a data de início,
            # é provável que o início ou fim da análise tenha sido após ou antes do expediente (9 as 17:30)
            # ou se os dois foram dentro do horário, só conta a diferença de tempo dentro do horário 
            # de expediente e não da diferença inteira de horas.
            if data_conclusao.date() > data_solicitacao.date():

                print('** data de fim da análise maior que de início **')

                # zerando a duracao no tipo timedelta    
                duracao = inicio_expediente - inicio_expediente    

                #VERIFICAR OS FERIADOS E OS DIAS QUE DEVEM SER ADICIONADOS:
                #PARA MAIS DE UM FERIADO, ADD IFS UM ABAIXO DO OUTRO
                if data_solicitacao.day == 24 or data_solicitacao.day == 31:
                    print('iniciou num feriado!')
                    add_days = '3 days' 

                # se a análise foi iniciada segunda,terça,quarta,quinta ou domingo 
                # só começa a contar no dia seguinte
                elif data_solicitacao.weekday() in [0,1,2,3] or data_solicitacao.weekday() == 6:
                    add_days = '1 days'

                # se a análise foi iniciada na sexta
                # só começa a contar a partir de segunda
                elif data_solicitacao.weekday() == 4:
                    add_days = '3 days'

                # se a análise foi iniciada no sábado
                elif data_solicitacao.weekday() == 5:
                    add_days = '2 days'

                # se a análise iniciou antes do fim do expediente
                # se não, precisamos verificar primeiro se iniciou antes do horário de expediente
                # para transformar no inicio do expediente, fazer a diferença entre o fim e o início
                # do expediente, adicionar a quantidade de dias e fazer a comparação.
                # se não iniciou, faz apenas a diferença
                # if data_solicitacao < fim_expediente:
                #     if data_solicitacao <= inicio_expediente:
                #         duracao = fim_expediente - inicio_expediente
                #     else:
                #         duracao = fim_expediente - data_solicitacao

                # agora, adicionamos os dias correspondentes ao inicio do expediente
                inicio_expediente = inicio_expediente + pd.Timedelta(add_days)

                # agora, calculamos a duração com a data de conclusão 
                # se a data de inicio for maior que a de fim, a duração é zero 
                # exemplo: iniciou na sexta e terminou no sábado, porém adicionamos três dias na sexta e 
                # ficou maior. se não, faz a diferença
                if inicio_expediente > data_conclusao:
                    duracao = pd.Timedelta('0 days')
                else:
                    duracao = duracao + (data_conclusao - inicio_expediente)
            
                print("Duração APÓS AJUSTE: " + str(duracao.days) + " dias / " + str(round(duracao.seconds/60)) + " minutos")

            # se a data de início for igual a data de fim
            else:

                print('** data de fim da análise igual de início **')

                # para fim de análise, consideramos fim de expediente 18hrs
                fim_expediente_conclusao = fim_expediente.replace(hour = 18)
                fim_expediente_conclusao = fim_expediente.replace(minute = 30)

                # se a analise foi iniciada e finalizada fora do horário de expediente, 
                # entao a duração é zero
                if (data_solicitacao < inicio_expediente and data_conclusao <= inicio_expediente) or (data_solicitacao > fim_expediente and data_conclusao > fim_expediente):
                    duracao = inicio_expediente - inicio_expediente 

                # se a analise foi iniciada antes do expediente, a duração é calculada 
                # começando de 9h (inicio do expediente)
                elif data_solicitacao < inicio_expediente and data_conclusao < fim_expediente_conclusao:
                    duracao = data_conclusao - inicio_expediente

                # se a analise foi finalizada depois do expediente, a duração é calculada 
                # com o horário do fim do expediente 
                elif (data_solicitacao >= inicio_expediente and data_solicitacao < fim_expediente) and data_conclusao > fim_expediente_conclusao:
                    duracao = fim_expediente_conclusao - data_solicitacao

            print('---------')
            print('')

            if(pd.notnull(duracao.days)):   
                
                #somando +1 ao total
                relatorio[tipo_renda]['TOTAL'] += 1
        
                #se não existe a chave da diferença de dias no dicionário
                if str(duracao.days) not in relatorio[tipo_renda]['TEMPO ATENDIMENTO DIAS']:

                    #se não existe, então cria
                    relatorio[tipo_renda]['TEMPO ATENDIMENTO DIAS'][str(duracao.days)] = 0

                #verificando intervalo de minutos da solicitacoes
                minutes = duracao.seconds/60
                                
                if minutes > 60 and tipo_renda=='LIMITE CARTÃO DE CRÉDITO':
                    print('Tipo da renda: ', tipo_renda)
                    print('Data de solicitacoes: ', data_solicitacao)
                    print('Data de conclusoes: ', data_conclusao)
                    print('minutos: ', minutes)
                    print('=====================')                    

                #adicionando +1 ao intervalo calculado
                if minutes <= 15:
                    relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 15'] += 1

                elif minutes > 15 and minutes <= 30:
                    relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 30'] += 1

                elif minutes > 30 and minutes <= 40:
                    relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 40'] += 1

                elif minutes > 40 and minutes <= 50:
                    relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 50'] += 1

                elif minutes > 50 and minutes <= 60:
                    relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 60'] += 1

                elif minutes > 60:
                    relatorio[tipo_renda]['TEMPO ATENDIMENTO MINUTOS']['MAIS DE 60'] += 1

                #se existe então soma +1 para contar quantas solicitações tem com aquela 
                #diferença de dias
                relatorio[tipo_renda]['TEMPO ATENDIMENTO DIAS'][str(math.floor(minutes/480))] += 1 

                #pegando o tempo de resolução da análise
                if tipo_renda not in duracoes:
                    duracoes[tipo_renda] = []
                duracoes[tipo_renda].append(duracao.seconds)
                duracoes['TOTAL'].append(duracao.seconds)

In [ ]:
csv_columns = ['TIPO RENDA', 'TOTAL', 'DIAS (0)', 'MIN. (<15)', 'MIN. (<30)', 'MIN. (<40)', 'MIN. (<50)', 'MIN. (<60)', 'MIN. (>60)', 'MÉDIA MIN.']

with open('relatorio-tempo-medio-legado.csv', 'w', newline='', encoding='utf-8') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    spamwriter.writerow(csv_columns)
    for key in relatorio:
        listaRow = list()
        listaRow.append(key)
      
        listaRow.append(relatorio[key]['TOTAL'])
        listaRow.append(relatorio[key]['TEMPO ATENDIMENTO DIAS']['0'])
        listaRow.append(relatorio[key]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 15'])
        listaRow.append(relatorio[key]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 30'])
        listaRow.append(relatorio[key]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 40'])
        listaRow.append(relatorio[key]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 50'])
        listaRow.append(relatorio[key]['TEMPO ATENDIMENTO MINUTOS']['ATÉ 60'])
        listaRow.append(relatorio[key]['TEMPO ATENDIMENTO MINUTOS']['MAIS DE 60'])
        listaRow.append(round(sum(duracoes[key])/len(duracoes[key])/60))
        
        spamwriter.writerow(listaRow)
        
print('Script executado com sucesso!')            